In [1]:
# !pip install obonet networkx --quiet

# import obonet
# import networkx as nx
# import pandas as pd
# import numpy as np
# import pickle
# from collections import defaultdict, deque
# from tqdm import tqdm

# # ============================================================================
# # 1. CẤU HÌNH ĐƯỜNG DẪN (CHECK KỸ)
# # ============================================================================
# CONFIG = {
#     # File OBO gốc
#     'OBO_FILE': "/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo",
    
#     'VOCAB_FILE': "/kaggle/input/c95-cafa6/vocab_C95_remove.csv",
#     'TARGET_FILE': "/kaggle/input/c95-cafa6/train_targets_C95.pkl",
    
#     # File đầu ra
#     'OUTPUT_PKL': "hierarchy_metadata_v2.pkl"
# }

# def build_metadata_v2_final():
#     print("🚀 BẮT ĐẦU TẠO METADATA V2 (FULL & CORRECTED)...")
    
#     # --- A. LOAD DỮ LIỆU ---
#     print("\n[1/4] Loading Graph & Vocab...")
#     graph = obonet.read_obo(CONFIG['OBO_FILE'])
#     vocab_df = pd.read_csv(CONFIG['VOCAB_FILE'])
#     vocab_terms = vocab_df['term'].tolist()
#     vocab_size = len(vocab_terms)
#     term_to_idx = {t: i for i, t in enumerate(vocab_terms)}
    
#     print(f"   - Graph nodes: {len(graph):,}")
#     print(f"   - Vocab size: {vocab_size:,}")

#     # --- B. TÍNH DEPTH NORM (BFS) ---
#     print("\n[2/4] Calculating Depth Norm...")
#     # (Phần này code cũ đã đúng, giữ nguyên logic BFS)
#     roots = ["GO:0008150", "GO:0003674", "GO:0005575"]
#     depth_dict = {node: float('inf') for node in graph.nodes()}
#     queue = deque()
    
#     for root in roots:
#         if root in graph:
#             depth_dict[root] = 0
#             queue.append(root)
            
#     while queue:
#         node = queue.popleft()
#         d = depth_dict[node]
#         # Predecessors = CON (Edge đi từ con -> cha)
#         for child in graph.predecessors(node):
#             if depth_dict[child] > d + 1:
#                 depth_dict[child] = d + 1
#                 queue.append(child)
                
#     depth_arr = np.array([depth_dict.get(t, 0) for t in vocab_terms], dtype=np.float32)
#     depth_arr[depth_arr == float('inf')] = 0.0
    
#     if depth_arr.max() > 0:
#         depth_norm = depth_arr / depth_arr.max()
#     else:
#         depth_norm = depth_arr
#     print(f"   - Max Depth: {depth_arr.max()}")

#     # --- C. TÍNH PARENT MAP (TRANSITIVE CLOSURE) - [NÂNG CẤP] ---
#     print("\n[3/4] Building Parent Map (Transitive - Fix Broken Chains)...")
    
#     # 1. Xây dựng đồ thị 'is_a' toàn vẹn từ file OBO gốc
#     # (Để tìm đường đi ngay cả khi node trung gian bị cắt)
#     full_isa_graph = nx.DiGraph()
    
#     print("   - Building full is_a graph...")
#     for node, data in graph.nodes(data=True):
#         if "is_a" in data:
#             for p_str in data["is_a"]:
#                 p_id = p_str.split(" ! ")[0]
#                 # Thêm cạnh Con -> Cha
#                 full_isa_graph.add_edge(node, p_id)
                
#     # 2. Map Con -> Tất cả Tổ tiên (Ancestors) có trong Vocab
#     child_to_parent = defaultdict(list)
#     count_edges = 0
#     vocab_set = set(vocab_terms) # Để tra cứu nhanh
    
#     for term, idx in tqdm(term_to_idx.items(), desc="Mapping Transitive"):
#         if term not in full_isa_graph: continue
        
#         # Tìm tất cả tổ tiên trong đồ thị gốc (bao gồm cả cha, ông, cụ...)
#         # nx.descendants trong DiGraph(Con->Cha) sẽ trả về tất cả Ancestors
#         try:
#             all_ancestors = nx.descendants(full_isa_graph, term)
#         except:
#             continue # Phòng trường hợp lỗi graph
            
#         # Chỉ giữ lại những tổ tiên CÓ MẶT trong Vocab
#         valid_ancestors = []
#         for anc in all_ancestors:
#             if anc in vocab_set:
#                 p_idx = term_to_idx[anc]
#                 valid_ancestors.append(p_idx)
                
#         if valid_ancestors:
#             child_to_parent[idx] = valid_ancestors
#             count_edges += len(valid_ancestors)

#     print(f"   - Mapped {count_edges:,} transitive relationships (Bridged gaps).")

#     # --- D. TÍNH IC NORM (INFORMATION CONTENT) - [SỬA QUAN TRỌNG] ---
#     print("\n[4/4] Calculating Information Content (IC)...")
    
#     # Load Targets
#     with open(CONFIG['TARGET_FILE'], 'rb') as f:
#         labels_dict = pickle.load(f)
    
#     # Đếm tần suất
#     term_counts = np.zeros(vocab_size, dtype=np.float32)
#     total_samples = len(labels_dict)
    
#     for indices in labels_dict.values():
#         if len(indices) > 0:
#             term_counts[indices] += 1
            
#     # Tính Frequency (Cộng epsilon nhỏ)
#     freq = (term_counts + 1e-9) / total_samples
    
#     # IC = -log(Freq)
#     ic_values = -np.log(freq)
    
#     # Normalize IC [0, 1]
#     ic_max = ic_values.max()
#     if ic_max > 0:
#         ic_norm = ic_values / ic_max
#     else:
#         ic_norm = np.zeros_like(ic_values)
        
#     print(f"   - IC Max Value: {ic_max:.4f}")
#     print(f"   - IC Norm Shape: {ic_norm.shape}")

#     # --- E. LƯU FILE ---
#     print(f"\n>>> Saving to {CONFIG['OUTPUT_PKL']}...")
    
#     save_data = {
#         'depth_norm': depth_norm,
#         'ic_norm': ic_norm,           # Đã có IC cho V4
#         'child_to_parent': dict(child_to_parent), # Đã fix logic is_a
#         'term_to_idx': term_to_idx
#     }
    
#     with open(CONFIG['OUTPUT_PKL'], 'wb') as f:
#         pickle.dump(save_data, f)
        
#     print("✅ DONE! Metadata V2 (Final) Created Successfully.")

# if __name__ == "__main__":
#     build_metadata_v2_final()

In [2]:
# import os
# import torch
# import torch.nn as nn
# import numpy as np
# import pandas as pd
# from torch.utils.data import Dataset, DataLoader
# from tqdm import tqdm
# import pickle
# import gc
# from collections import OrderedDict
# from torch.cuda.amp import autocast

# # ============================================================================
# # 1. CẤU HÌNH SUPER-BOOST V4
# # ============================================================================
# CONFIG = {
#     # --- Model ---
#     'input_dim': 1280,
#     'hidden_dims': [2048, 4096],
#     'dropout': 0.3,
#     'num_classes': 6413, # C95_remove

#     # --- System ---
#     'device': 'cuda' if torch.cuda.is_available() else 'cpu',
#     'batch_size': 128, 
#     'num_workers': 4,

#     # --- Paths ---
#     'MODEL_PATH': "/kaggle/input/model-cafa6/best_model_wide_0.63.pth",
#     'EMBED_DIR': "/kaggle/input/cafa6-embeds",
#     'METADATA_PKL': "/kaggle/working/hierarchy_metadata_v2.pkl", # File V2 vừa tạo
#     'VOCAB_FILE': "/kaggle/input/c95-cafa6/vocab_C95_remove.csv",

#     # --- THAM SỐ V4 (TUNED BY TOP KAGGLERS) ---
#     'alpha': 0.15,       # Depth Boost
#     'beta':  0.35,       # IC Boost (Quan trọng nhất)
#     'gamma': 0.05,       # Adaptive Score Boost
    
#     'threshold': 0.01,   # Ngưỡng thấp để đón nhãn sâu
#     'top_k_leaf': 120,    
#     'final_cap': 200,   
    
#     # --- Root Terms ---
#     'ROOT_TERMS': ["GO:0003674", "GO:0005575", "GO:0008150"]
# }

# print(f"🚀 Config V4: Alpha={CONFIG['alpha']} | Beta={CONFIG['beta']} | Gamma={CONFIG['gamma']}")

# # ============================================================================
# # 2. CLASS MODEL & DATASET
# # ============================================================================
# class WideProteinMLP(nn.Module):
#     def __init__(self, input_dim, num_classes, hidden_dims=[2048, 4096], dropout=0.3):
#         super().__init__()
#         layers = [nn.LayerNorm(input_dim)]
#         prev = input_dim
#         for h in hidden_dims:
#             layers += [nn.Linear(prev, h), nn.GELU(), nn.Dropout(dropout)]
#             prev = h
#         layers.append(nn.Linear(prev, num_classes))
#         self.net = nn.Sequential(*layers)
#     def forward(self, x): return self.net(x)

# class TestDataset(Dataset):
#     def __init__(self, embed_dir):
#         with open(os.path.join(embed_dir, "test_ids.txt")) as f:
#             self.ids = [line.strip() for line in f]
#         self.embed_matrix = np.load(os.path.join(embed_dir, "test_embeds.npy"), mmap_mode="r")
#     def __len__(self): return len(self.ids)
#     def __getitem__(self, idx):
#         return torch.from_numpy(self.embed_matrix[idx].copy()).float(), self.ids[idx]

# # ============================================================================
# # 3. LOGIC XỬ LÝ V4 (CORE)
# # ============================================================================

# def process_single_protein_v4_fixed(scores, depth_norm, ic_norm, parent_map, vocab_terms):
#     """
#     Fixed & Robust Version:
#     1. Select Candidates từ Base Scores (Tránh boost rác).
#     2. Robust Index Mapping (Tránh lỗi sai điểm).
#     3. Safe Boosting (Clip 1.5 & Protect High Scores).
#     4. Standard Max Propagation.
#     5. Root Fallback.
#     """
    
#     # 1. Select Candidates từ Base Scores
#     # Chỉ xem xét Top-K lá có điểm thô cao nhất để tiết kiệm tính toán và lọc nhiễu
#     n_terms = len(scores)
#     if CONFIG['top_k_leaf'] >= n_terms:
#         cand_idx = np.arange(n_terms)
#     else:
#         # Lấy top K điểm cao nhất (Raw)
#         # argsort trả về index tăng dần -> lấy đuôi
#         cand_idx = np.argsort(scores)[-CONFIG['top_k_leaf']:]
        
#     # 2. Compute Boost (Chỉ trên tập Candidate)
#     cand_scores = scores[cand_idx]
#     cand_depth = depth_norm[cand_idx]
#     cand_ic = ic_norm[cand_idx]
    
#     # Công thức V4
#     bf = 1.0 + CONFIG['alpha'] * cand_depth + CONFIG['beta'] * cand_ic + CONFIG['gamma'] * (cand_depth * cand_scores)
    
#     # Safety Clip: Boost tối đa 1.5 lần (V7 Gentle)
#     MAX_BOOST = 1.5
#     bf = np.minimum(bf, MAX_BOOST)
    
#     # High score protection: Score gốc > 0.8 giữ nguyên
#     high_mask = cand_scores > 0.8
#     bf[high_mask] = 1.0
    
#     boosted_cand = cand_scores * bf
#     boosted_cand = np.clip(boosted_cand, 0.0, 1.0)
    
#     # 3. Filter Threshold
#     # Chỉ giữ lại những nhãn đã boost vượt qua ngưỡng
#     keep_mask = boosted_cand >= CONFIG['threshold']
    
#     # [ROBUST MAPPING] Tạo map từ Global Index -> Boosted Score
#     # Để tra cứu nhanh và chính xác
#     # cand_idx: Global Indices của các candidate
#     # boosted_cand: Scores tương ứng
#     score_lookup = dict(zip(cand_idx, boosted_cand))
    
#     # Lấy danh sách Global Index vượt qua ngưỡng
#     kept_global_idx = cand_idx[keep_mask]
    
#     # Fallback: Nếu lọc xong không còn gì, lấy Top 1 tốt nhất (để không rỗng)
#     if len(kept_global_idx) == 0:
#         top1_local = np.argsort(boosted_cand)[-1]
#         kept_global_idx = np.array([cand_idx[top1_local]])
    
#     # 4. Hierarchy Repair (Max Propagation)
#     final_scores = {}
    
#     # Khởi tạo điểm cho các node lá được chọn
#     for idx in kept_global_idx:
#         final_scores[idx] = score_lookup[idx]
        
#     queue = list(final_scores.keys())
#     processed = set(queue)
#     idx_ptr = 0
    
#     while idx_ptr < len(queue):
#         cur_idx = queue[idx_ptr]; idx_ptr += 1
#         cur_score = final_scores[cur_idx]
        
#         parents = parent_map.get(cur_idx, [])
#         for p_idx in parents:
#             # Lấy điểm gốc của cha (nếu có trong candidate thì lấy boosted, ko thì lấy raw)
#             # Logic: Nếu cha nằm trong candidate list thì dùng điểm đã boost, 
#             # nếu không (cha điểm thấp quá ko lọt top) thì dùng điểm raw gốc.
#             if p_idx in score_lookup:
#                 p_base = score_lookup[p_idx]
#             else:
#                 p_base = float(scores[p_idx]) if p_idx < n_terms else 0.0
                
#             p_prev = final_scores.get(p_idx, p_base)
            
#             # QUY TẮC MAX CHUẨN: Cha = Max(Cha cũ, Con)
#             new_score = max(p_prev, cur_score)
            
#             # Chỉ lan truyền nếu điểm tăng đáng kể (> 1e-6)
#             if new_score > p_prev + 1e-6:
#                 final_scores[p_idx] = new_score
#                 if p_idx not in processed:
#                     queue.append(p_idx)
#                     processed.add(p_idx)
#             # Đảm bảo cha luôn có mặt (Consistency)
#             elif p_idx not in final_scores:
#                 final_scores[p_idx] = p_prev
#                 if p_idx not in processed:
#                     queue.append(p_idx)
#                     processed.add(p_idx)

#     # 5. Final Cap & Format
#     # Sort giảm dần
#     sorted_items = sorted(final_scores.items(), key=lambda x: x[1], reverse=True)
    
#     # Cắt Top N (Final Cap)
#     if len(sorted_items) > CONFIG['final_cap']:
#         sorted_items = sorted_items[:CONFIG['final_cap']]
    
#     results = []
#     for idx, score in sorted_items:
#         t_id = vocab_terms[idx]
#         results.append((t_id, float(score)))
        
#     # 6. Root Fallback (Chỉ thêm nếu danh sách rỗng)
#     existing_terms = set([t for t,_ in results])
#     for root in CONFIG['ROOT_TERMS']:
#         if root not in existing_terms:
#             results.append((root, 0.03))  # score thấp an toàn
#             existing_terms.add(root)
            
#     return results, existing_terms

# # ============================================================================
# # 4. MAIN INFERENCE
# # ============================================================================
# def run_inference_v4():
#     print(">>> 1. Loading Metadata V2...")
#     vocab_df = pd.read_csv(CONFIG['VOCAB_FILE'])
#     vocab_terms = np.array(vocab_df['term'].tolist())
    
#     # Load Metadata có IC
#     with open(CONFIG['METADATA_PKL'], 'rb') as f:
#         meta = pickle.load(f)
#         depth_norm = meta['depth_norm']
#         ic_norm = meta['ic_norm']          # <--- Key mới
#         parent_map = meta['child_to_parent']
        
#     print(f"   - Metadata loaded. IC shape: {ic_norm.shape}")

#     print(">>> 2. Loading Model...")
#     model = WideProteinMLP(CONFIG['input_dim'], CONFIG['num_classes'], CONFIG['hidden_dims'], CONFIG['dropout'])
    
#     # Load weights
#     ckpt = torch.load(CONFIG['MODEL_PATH'], map_location="cpu")
#     sd = ckpt['model_state'] if isinstance(ckpt, dict) and 'model_state' in ckpt else ckpt
#     new_sd = OrderedDict()
#     for k, v in sd.items():
#         new_sd[k.replace("module.", "")] = v
#     model.load_state_dict(new_sd)

#     # 2 GPU setup
#     model.to(CONFIG['device'])
#     if torch.cuda.device_count() > 1:
#         print(f"🔥 Active GPUs: {torch.cuda.device_count()}")
#         model = nn.DataParallel(model)
#     model.eval()

#     ds = TestDataset(CONFIG['EMBED_DIR'])
#     dl = DataLoader(ds, batch_size=CONFIG['batch_size'], shuffle=False, 
#                     num_workers=4, pin_memory=True)

#     OUT_FILE = "submission_v1.tsv"
#     f_out = open(OUT_FILE, "w")
#     print(f">>> 3. Streaming Inference to {OUT_FILE}...")
    
#     count = 0
#     with torch.no_grad():
#         for features, prot_ids in tqdm(dl):
#             features = features.to(CONFIG['device'])
            
#             with autocast():
#                 logits = model(features)
#                 probs = torch.sigmoid(logits)

        
#             probs_np = probs.float().cpu().numpy()
            
#             for i, pid in enumerate(prot_ids):
#                 results, existing_terms = process_single_protein_v4_fixed(
#                     probs_np[i], depth_norm, ic_norm, parent_map, vocab_terms
#                 )

#                 # Ensure root terms present (already handled in function, but double-check harmless)
#                 for root in CONFIG["ROOT_TERMS"]:
#                     if root not in existing_terms:
#                         results.append((root, 0.03))
#                         existing_terms.add(root)

#                 for term, score in results:
#                     f_out.write(f"{pid}\t{term}\t{score:.3f}\n")
#                     count += 1
                        
#     f_out.close()
    
#     size_mb = os.path.getsize(OUT_FILE) / (1024**2)
#     print(f"\n✅ DONE! Submission V4 Created.")
#     print(f"   - Total rows: {count:,}")
#     print(f"   - File size: {size_mb:.2f} MB")

# if __name__ == "__main__":
#     run_inference_v4()

In [3]:
# import pandas as pd
# import os
# from tqdm import tqdm

# INPUT_FILE = "submission_v1.tsv"  # File kết quả từ config Strong Boost
# OUTPUT_FILE = "submission.tsv"

# # CẤU HÌNH CẮT GỌT DỰA TRÊN BẢNG PHÂN TÍCH
# CUT_THRESHOLD = 0.10  # Bắt đầu lấy từ vùng IA=1.48
# CUT_LIMIT = 150       # Giữ Top 150 (Thay vì 200) để tăng Precision

# def optimize_submission():
#     print(f"🔥 Optimizing {INPUT_FILE}...")
#     if os.path.exists(OUTPUT_FILE): os.remove(OUTPUT_FILE)
    
#     chunksize = 5_000_000
#     total_kept = 0
    
#     with pd.read_csv(INPUT_FILE, sep='\t', names=['P', 'T', 'S'], chunksize=chunksize) as reader:
#         for chunk in tqdm(reader):
#             # 1. Lọc ngưỡng 0.10
#             filtered = chunk[chunk['S'] >= CUT_THRESHOLD]
            
#             # 2. Top 150
#             filtered = filtered.sort_values(['P', 'S'], ascending=[True, False])
#             filtered = filtered.groupby('P').head(CUT_LIMIT)
            
#             if len(filtered) > 0:
#                 filtered.to_csv(OUTPUT_FILE, sep='\t', header=False, index=False, mode='a')
#                 total_kept += len(filtered)

#     print(f"✅ DONE! New file: {OUTPUT_FILE}")
#     print(f"   Rows: {total_kept:,}")
#     print(f"   Density: {total_kept/224309:.1f} labels/protein")

# if __name__ == "__main__":
#     optimize_submission()

In [4]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

# # --- NÊN ĐỔI TÊN FILE CHO ĐÚNG ---
# # base_path   = "/kaggle/input/sample-submission/0.275_submission.tsv"
# # boost350    = "/kaggle/input/sample-submission/0.229_submission.tsv"
# # boost50     = "/kaggle/input/sample-submission/0.233_submission.tsv"
# # low     = "/kaggle/input/sample-submission/0.266_submission.tsv"

# submit = "/kaggle/working/submission.tsv"

# def load_sub(path):
#     return pd.read_csv(path, sep='\t', names=['protein','term','score'])

In [5]:
# import pandas as pd
# import numpy as np
# from tqdm import tqdm
# import os
# from collections import defaultdict

# # =========================================================
# # CẤU HÌNH & THAM SỐ
# # =========================================================
# SUB_FILE = "submission.tsv"
# IA_FILE  = "/kaggle/input/cafa-6-protein-function-prediction/IA.tsv"
# CHUNK_SIZE = 5_000_000 
# ROOT_TERMS = ["GO:0003674", "GO:0005575", "GO:0008150"]
# SCORE_BINS = np.linspace(0.0, 1.0, 11) # Tạo 11 mốc: 0.0, 0.1, 0.2, ..., 1.0

# def analyze_ia_extremes_fixed():
#     # 1. Load submission (Phát hiện cột)
#     print("1. Reading submission to identify columns...")
#     df_head = pd.read_csv(SUB_FILE, sep=None, engine="python", header=None, nrows=1000)

#     go_col, score_col = None, None
#     for col in df_head.columns:
#         sample = df_head[col].astype(str)
#         if sample.str.startswith("GO:").any(): go_col = col
#         if pd.to_numeric(sample, errors="coerce").notna().mean() > 0.9: score_col = col

#     if go_col is None or score_col is None:
#         raise ValueError("Không tìm thấy cột GO-term hoặc Score. Kiểm tra lại SUB_FILE.")

#     # 2. Load IA Map
#     ia_df = pd.read_csv(IA_FILE, sep="\t", header=None, names=["go", "ia"])
#     ia_map = dict(zip(ia_df.go, ia_df.ia))
#     print(f"2. IA Map Loaded ({len(ia_map):,} terms).")

#     # 3. PHÂN TÍCH STREAMING
#     ia_bin_stats = defaultdict(lambda: {'total_ia': 0.0, 'count': 0})

#     print("\n3. Starting Streaming Analysis (Binning Mean IA)...")
#     reader = pd.read_csv(SUB_FILE, sep=None, engine="python", header=None, chunksize=CHUNK_SIZE)

#     for chunk in tqdm(reader, desc="Processing Chunks"):
#         # Đổi tên cột
#         df_chunk = chunk.rename(columns={go_col: "go", score_col: "score"})
        
#         # Lọc 3 nhãn gốc
#         df_chunk = df_chunk[~df_chunk['go'].isin(ROOT_TERMS)].copy()
        
#         # Gán IA (Fill NaN = 0 cho an toàn)
#         df_chunk["ia"] = df_chunk["go"].map(ia_map).fillna(0.0)
        
#         # Lọc điểm số hợp lệ
#         df_chunk = df_chunk[(df_chunk['score'] >= 0) & (df_chunk['score'] <= 1.0)].copy()

#         # Tính toán BINS
#         df_chunk['bin_index'] = np.digitize(df_chunk['score'], bins=SCORE_BINS) - 1 # Đổi về index 0-9
        
#         # Nhóm theo index bin và tính tổng IA và Count
#         chunk_stats = df_chunk.groupby('bin_index').agg(
#             total_ia=('ia', 'sum'),
#             count=('ia', 'count')
#         ).reset_index()

#         # Cập nhật global stats
#         for _, row in chunk_stats.iterrows():
#             # [FIX LỖI] Ép kiểu về int tiêu chuẩn
#             bin_idx = int(row['bin_index']) 
            
#             # Kiểm tra lỗi ngoài biên (do np.digitize đôi khi trả về 10)
#             if bin_idx >= len(SCORE_BINS) - 1 or bin_idx < 0:
#                 continue 
            
#             bin_min = SCORE_BINS[bin_idx]
#             bin_max = SCORE_BINS[bin_idx + 1]
            
#             key = (bin_min, bin_max)
#             ia_bin_stats[key]['total_ia'] += row['total_ia']
#             ia_bin_stats[key]['count'] += row['count']


#     # --- KẾT QUẢ CUỐI CÙNG ---
#     print("\n" + "="*70)
#     print(f"📊 PHÂN TÍCH MEAN IA THEO KHOẢNG ĐIỂM SỐ")
#     print("="*70)
    
#     total_all_counts = sum(s['count'] for s in ia_bin_stats.values())
    
#     print(f"{'Score Range':<15} | {'Mean IA':<10} | {'Total Count':<15} | {'Density (%)':<10}")
#     print("-" * 70)

#     for (bin_min, bin_max), stats in sorted(ia_bin_stats.items()):
#         count = stats['count']
#         mean_ia = stats['total_ia'] / count if count > 0 else 0.0
#         density = (count / total_all_counts) * 100 if total_all_counts > 0 else 0.0
        
#         range_str = f"[{bin_min:.2f} - {bin_max:.2f})"
        
#         print(f"{range_str:<15} | {mean_ia:<10.4f} | {count:<15,} | {density:<9.1f}%")

#     print("="*70)

# if __name__ == "__main__":
#     analyze_ia_extremes_fixed()

In [6]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

# # File bạn vừa tạo
# SUBMISSION_FILE = "submission.tsv"

# def analyze_threshold_impact():
#     print(f"📂 Đang đọc file {SUBMISSION_FILE}...")
#     # Đọc file (cột: Protein, Term, Score)
#     df = pd.read_csv(SUBMISSION_FILE, sep='\t', names=['Protein', 'Term', 'Score'])
    
#     total_rows = len(df)
#     total_proteins = df['Protein'].nunique()
    
#     print(f"✅ Đã tải xong!")
#     print(f"   - Tổng dòng hiện tại: {total_rows:,}")
#     print(f"   - Tổng Protein: {total_proteins:,}")
#     print(f"   - Trung bình nhãn/Protein (Hiện tại): {total_rows / total_proteins:.1f}")
    
#     # --- THỐNG KÊ NGƯỠNG GIẢ LẬP ---
#     print("\n📊 NẾU TĂNG THRESHOLD THÌ SAO?")
#     print("-" * 50)
#     print(f"{'Threshold':<10} | {'Số dòng còn lại':<15} | {'Nhãn/Protein':<15} | {'% Giữ lại':<10}")
#     print("-" * 50)
    
#     # Các mốc threshold muốn thử
#     thresholds = [0.01, 0.05, 0.08, 0.10, 0.12, 0.15, 0.20]
    
#     for thr in thresholds:
#         # Lọc giả lập
#         filtered_count = (df['Score'] >= thr).sum()
#         avg_labels = filtered_count / total_proteins
#         percent_kept = (filtered_count / total_rows) * 100
        
#         print(f"{thr:<10.2f} | {filtered_count:<15,} | {avg_labels:<15.1f} | {percent_kept:<9.1f}%")
        
#     print("-" * 50)
    
#     # --- VẼ BIỂU ĐỒ PHÂN BỐ SCORE ---
#     plt.figure(figsize=(10, 5))
#     sns.histplot(df['Score'], bins=100, color='teal', kde=False)
#     plt.axvline(0.08, color='red', linestyle='--', label='Current Threshold (0.05)')
#     plt.title(f'Phân bố điểm số trong file {SUBMISSION_FILE}')
#     plt.xlabel('Score')
#     plt.ylabel('Số lượng')
#     plt.legend()
#     plt.yscale('log') # Dùng log scale để nhìn rõ vùng thấp
#     plt.show()

# analyze_threshold_impact()

In [7]:
import os
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import gc
import torch.nn as nn
from collections import OrderedDict
from torch import amp

class WideProteinMLP(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dims=[2048, 4096], dropout=0.3):
        super().__init__()
        layers = []
        # Input Norm: Giúp ổn định đầu vào từ Embeddings
        layers.append(nn.LayerNorm(input_dim)) 
        
        prev = input_dim
        for h in hidden_dims:
            layers.append(nn.Linear(prev, h))
            layers.append(nn.GELU())   # Activation hiện đại
            layers.append(nn.Dropout(dropout))
            prev = h
            
        layers.append(nn.Linear(prev, num_classes))
        self.net = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.net(x)

# ============================================================================
# 1. CẤU HÌNH INFERENCE
# ============================================================================
CONFIG = {
    # --- Model Architecture (Phải khớp với lúc Train) ---
    'input_dim': 1280,             # ESM2-t33 embeddings
    'hidden_dims': [2048, 4096],   # Wide MLP layers
    'dropout': 0.3,                # Dropout 
    'num_classes': 6413,           # Số lượng nhãn của bộ C95_remove 
    
    # --- System ---
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'batch_size': 64,            
    'num_workers': 2,
    
    # --- Paths ---
    'MODEL_PATH': "/kaggle/input/model-cafa6/best_model_wide_0.63.pth",  # Đường dẫn file checkpoint đã lưu
    'EMBED_DIR': "/kaggle/input/cafa6-embeds", 
    'VOCAB_FILE': "/kaggle/input/c95-cafa6/vocab_C95_remove.csv", # File vocab khớp với model
    
    # --- Post-processing ---
    'min_score_threshold': 0.2,   # Chỉ lấy các dự đoán có xác suất >= 1%
    'submission_limit': 150       # Giới hạn tối đa 1500 nhãn/protein
}

print(f"🚀 Cấu hình Inference: Device={CONFIG['device']} | Threshold={CONFIG['min_score_threshold']}")

# ============================================================================
# 2. ĐỊNH NGHĨA DATASET CHO TẬP TEST
# ============================================================================
class TestDataset(Dataset):
    def __init__(self, embed_dir):
        with open(os.path.join(embed_dir, "test_ids.txt")) as f:
            self.ids = [line.strip() for line in f]
        # mmap là chuẩn để tiết kiệm RAM hệ thống
        self.embed_matrix = np.load(os.path.join(embed_dir, "test_embeds.npy"), mmap_mode="r")

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        # NÊN dùng .copy() ở đây. 
        # Lý do: Batch size 64 chỉ tốn vài KB RAM, nhưng giúp GPU không phải chờ ổ cứng đọc dữ liệu.
        feat = torch.from_numpy(self.embed_matrix[idx].copy()).float()
        return feat, self.ids[idx]

# ============================================================================
# 3. HÀM CHẠY DỰ ĐOÁN (INFERENCE LOOP)
# ============================================================================
def run_inference_gpu_stream():
    # 1. Setup
    print(f"Loading vocab from {CONFIG['VOCAB_FILE']}...")
    vocab_terms = pd.read_csv(CONFIG['VOCAB_FILE'])['term'].values
    
    # Tăng num_workers lên 4 nếu dùng 2 GPU để đẩy dữ liệu nhanh hơn
    ds = TestDataset(CONFIG['EMBED_DIR'])
    dl = DataLoader(ds, batch_size=CONFIG['batch_size'], shuffle=False, 
                    num_workers=4, pin_memory=True) # num_workers=4

    model = WideProteinMLP(CONFIG['input_dim'], CONFIG['num_classes'], 
                           CONFIG['hidden_dims'], CONFIG['dropout'])
    
    # 2. Load Weights (Clean 'module.' prefix)
    print(f"Loading model from {CONFIG['MODEL_PATH']}...")
    ckpt = torch.load(CONFIG['MODEL_PATH'], map_location="cpu")
    sd = ckpt['model_state'] if isinstance(ckpt, dict) and 'model_state' in ckpt else ckpt
    clean_sd = {k.replace("module.", ""): v for k, v in sd.items()}
    model.load_state_dict(clean_sd)
    
    # 3. KÍCH HOẠT 2 GPU (QUAN TRỌNG)
    model.to(CONFIG['device'])
    if torch.cuda.device_count() > 1:
        print(f"🔥 Kích hoạt {torch.cuda.device_count()} GPUs!")
        model = nn.DataParallel(model) # <--- Dòng này giúp chạy trên cả 2 GPU
    
    model.eval()

    # 4. Inference Loop
    out_file = "submission.tsv"
    print(f"Streaming predictions to {out_file}...")
    f = open(out_file, "w")
    
    count = 0
    with torch.no_grad():
        for features, prot_ids in tqdm(dl):
            features = features.to(CONFIG['device'])
            
            # Autocast: Giúp giảm VRAM, tăng tốc độ trên T4
            with amp.autocast('cuda'):
                logits = model(features)
                probs = torch.sigmoid(logits)
            
            # Chuyển về CPU ngay lập tức để giải phóng GPU
            probs = probs.float().cpu().numpy()
            
            # Xử lý kết quả (CPU làm việc này trong khi GPU tính batch tiếp theo)
            for i, pid in enumerate(prot_ids):
                p = probs[i]
                
                # Lọc nhanh bằng numpy
                idxs = np.where(p >= CONFIG['min_score_threshold'])[0]
                if len(idxs) == 0: continue
                
                scores = p[idxs]
                
                # Cắt Top K
                if len(idxs) > CONFIG['submission_limit']:
                    # partition nhanh hơn argsort toàn bộ
                    top_k_indices = np.argpartition(scores, -CONFIG['submission_limit'])[-CONFIG['submission_limit']:]
                    idxs = idxs[top_k_indices]
                    scores = scores[top_k_indices]
                
                # Ghi file
                batch_terms = vocab_terms[idxs]
                for term, sc in zip(batch_terms, scores):
                    f.write(f"{pid}\t{term}\t{sc:.3f}\n")
                    count += 1
            
            # Dọn dẹp thủ công (Optional, Python tự lo được nhưng thêm cho chắc)
            del features, logits, probs
            
    f.close()
    print(f"✅ DONE! Created submission.tsv with {count:,} rows.")

if __name__ == "__main__":
    run_inference_gpu_stream()

🚀 Cấu hình Inference: Device=cuda | Threshold=0.2
Loading vocab from /kaggle/input/c95-cafa6/vocab_C95_remove.csv...
Loading model from /kaggle/input/model-cafa6/best_model_wide_0.63.pth...
🔥 Kích hoạt 2 GPUs!
Streaming predictions to submission.tsv...


100%|██████████| 3505/3505 [01:28<00:00, 39.46it/s]


✅ DONE! Created submission.tsv with 32,761,135 rows.
